In [ ]:
tornado_gdf.crs

# Wrap Shapely geometry in a GeoSeries with CRS
river_crs = gpd.GeoSeries([river_il_geom], crs=target_crs).crs

# Now reproject tornadoes to the same CRS
tornado_gdf = tornado_gdf.to_crs(river_crs)

print(tornado_gdf.crs)  # Should be EPSG:26915 (UTM)

# --- Step 1: Define helper to compute nearest distance to river geometry ---
def nearest_distance_km(point, river_geom):
    """Compute nearest distance from a shapely Point to river_geom in km"""
    nearest_pt = nearest_points(point, river_geom)[1]
    return point.distance(nearest_pt) / 1000  # meters → km

# --- Step 2: Ensure we have start/end points as Point geometries ---
# For LineStrings, start = first point, end = last point
def start_point(geom):
    if geom.geom_type == 'LineString':
        return Point(geom.coords[0])
    elif geom.geom_type == 'Point':
        return geom
    return None

def end_point(geom):
    if geom.geom_type == 'LineString':
        return Point(geom.coords[-1])
    elif geom.geom_type == 'Point':
        return geom
    return None

tornado_gdf['start_pt'] = tornado_gdf['geometry'].apply(start_point)
tornado_gdf['end_pt'] = tornado_gdf['geometry'].apply(end_point)

# --- Step 3: Assign season ---
def get_season(month):
    if month in [12,1,2]: return 'DJF'
    if month in [3,4,5]: return 'MAM'
    if month in [6,7,8]: return 'JJA'
    return 'SON'

tornado_gdf['season'] = tornado_gdf['date'].dt.month.apply(get_season)

# --- Step 4: Filter only tornadoes crossing/buffer ---
river_buffer_tornadoes = tornado_gdf[tornado_gdf.intersects(river_buffer)].copy()

# Assign side_type using centroids relative to river center x
river_center_x = river_il_geom.centroid.x
river_buffer_tornadoes['centroid'] = river_buffer_tornadoes['geometry'].centroid
river_buffer_tornadoes.loc[river_buffer_tornadoes['centroid'].x < river_center_x, 'side_type'] = 'West-side'
river_buffer_tornadoes.loc[river_buffer_tornadoes['centroid'].x >= river_center_x, 'side_type'] = 'East-side'
river_buffer_tornadoes.loc[river_buffer_tornadoes.intersects(river_il_geom), 'side_type'] = 'Crossed River'

# --- Step 5: Compute distances for start and end points ---
river_buffer_tornadoes['start_dist_km'] = river_buffer_tornadoes['start_pt'].apply(lambda p: nearest_distance_km(p, river_il_geom))
river_buffer_tornadoes['end_dist_km']   = river_buffer_tornadoes['end_pt'].apply(lambda p: nearest_distance_km(p, river_il_geom))

# Quick spot check
print(river_buffer_tornadoes[['side_type','start_dist_km','end_dist_km','season']].head())

# --- Step 6: Build 4x5 table ---
rows = ['Start West', 'Start East', 'End West', 'End East']
cols = ['DJF', 'MAM', 'JJA', 'SON', 'Annual']
side_order = ['West-side','Crossed River','East-side']

# Initialize empty table dictionary
table_data = {row: {col: [] for col in cols} for row in rows}

# Compute mean distances per season and side
for season in ['DJF','MAM','JJA','SON']:
    seasonal = river_buffer_tornadoes[river_buffer_tornadoes['season']==season]
    
    # Start West
    table_data['Start West'][season] = [
        round(seasonal[seasonal['side_type']=='West-side']['start_dist_km'].mean(),1) if not seasonal[seasonal['side_type']=='West-side'].empty else None,
        round(seasonal[seasonal['side_type']=='Crossed River']['start_dist_km'].mean(),1) if not seasonal[seasonal['side_type']=='Crossed River'].empty else None,
        None  # East-side cannot have start west
    ]
    # Start East
    table_data['Start East'][season] = [
        None,
        round(seasonal[seasonal['side_type']=='Crossed River']['start_dist_km'].mean(),1) if not seasonal[seasonal['side_type']=='Crossed River'].empty else None,
        round(seasonal[seasonal['side_type']=='East-side']['start_dist_km'].mean(),1) if not seasonal[seasonal['side_type']=='East-side'].empty else None
    ]
    # End West
    table_data['End West'][season] = [
        round(seasonal[seasonal['side_type']=='West-side']['end_dist_km'].mean(),1) if not seasonal[seasonal['side_type']=='West-side'].empty else None,
        round(seasonal[seasonal['side_type']=='Crossed River']['end_dist_km'].mean(),1) if not seasonal[seasonal['side_type']=='Crossed River'].empty else None,
        None
    ]
    # End East
    table_data['End East'][season] = [
        None,
        round(seasonal[seasonal['side_type']=='Crossed River']['end_dist_km'].mean(),1) if not seasonal[seasonal['side_type']=='Crossed River'].empty else None,
        round(seasonal[seasonal['side_type']=='East-side']['end_dist_km'].mean(),1) if not seasonal[seasonal['side_type']=='East-side'].empty else None
    ]

# Annual column
annual = river_buffer_tornadoes
for row in rows:
    if 'Start West' in row:
        table_data[row]['Annual'] = [
            round(annual[annual['side_type']=='West-side']['start_dist_km'].mean(),1) if not annual[annual['side_type']=='West-side'].empty else None,
            round(annual[annual['side_type']=='Crossed River']['start_dist_km'].mean(),1) if not annual[annual['side_type']=='Crossed River'].empty else None,
            None
        ]
    elif 'Start East' in row:
        table_data[row]['Annual'] = [
            None,
            round(annual[annual['side_type']=='Crossed River']['start_dist_km'].mean(),1) if not annual[annual['side_type']=='Crossed River'].empty else None,
            round(annual[annual['side_type']=='East-side']['start_dist_km'].mean(),1) if not annual[annual['side_type']=='East-side'].empty else None
        ]
    elif 'End West' in row:
        table_data[row]['Annual'] = [
            round(annual[annual['side_type']=='West-side']['end_dist_km'].mean(),1) if not annual[annual['side_type']=='West-side'].empty else None,
            round(annual[annual['side_type']=='Crossed River']['end_dist_km'].mean(),1) if not annual[annual['side_type']=='Crossed River'].empty else None,
            None
        ]
    elif 'End East' in row:
        table_data[row]['Annual'] = [
            None,
            round(annual[annual['side_type']=='Crossed River']['end_dist_km'].mean(),1) if not annual[annual['side_type']=='Crossed River'].empty else None,
            round(annual[annual['side_type']=='East-side']['end_dist_km'].mean(),1) if not annual[annual['side_type']=='East-side'].empty else None
        ]

# --- Step 7: Display as DataFrame with color-coded cells ---
import pandas as pd
from IPython.display import display

# Cleaner formatting helper
def format_cell(vals):
    if vals is None: 
        return ""
    # Replace None with dash
    return ", ".join([f"{v:.1f}" if v is not None else "-" for v in vals])

# Build DataFrame
df_table = pd.DataFrame({col: [format_cell(table_data[row][col]) for row in rows] for col in cols}, index=rows)

# Color-coding for 3 categories
def color_code(val):
    # Split by comma and assign colors
    parts = val.split(", ")
    colors = ['#1f78b4', '#33a02c', '#e31a1c']  # West, Crossed, East
    styled_parts = []
    for i, part in enumerate(parts):
        if part != "-":
            styled_parts.append(f'<span style="color:{colors[i]};font-weight:bold">{part}</span>')
        else:
            styled_parts.append(part)
    return ", ".join(styled_parts)

df_styled = df_table.style.format(lambda v: color_code(v))
display(df_styled)